In [1]:
import json
import numpy as np
import sqlalchemy
import pandas as pd
from fastapi import FastAPI, HTTPException
from langchain import LLMChain
from langchain.llms import Ollama
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate

In [2]:
# JSON Configuration and Metadata Data Embedded within the Script
db_config = {
    "dialect": "mysql",
    "username": "root",
    "password": "password",
    "host": "localhost",
    "port": 3306,
    "database": "otl_erp"
}

In [3]:
metadata = {
    "tables": {
        "products": {
            "columns": {
                "id": "integer",
                "product_name": "string",
                "category_id": "integer",
                "price": "decimal",
                "created_at": "datetime"
            },
            "primary_key": ["id"],
            "description": "Contains information about products in the inventory."
        },
        "categories": {
            "columns": {
                "id": "integer",
                "category_name": "string",
                "description": "string"
            },
            "primary_key": ["id"],
            "description": "Contains product category information."
        },
        "sales": {
            "columns": {
                "id": "integer",
                "product_id": "integer",
                "quantity": "integer",
                "amount": "decimal",
                "sale_date": "datetime"
            },
            "primary_key": ["id"],
            "description": "Records each sale transaction with the product sold and the amount."
        },
        "imports": {
            "columns": {
                "id": "integer",
                "product_id": "integer",
                "quantity": "integer",
                "import_date": "datetime",
                "supplier": "string"
            },
            "primary_key": ["id"],
            "description": "Contains records of imported products."
        }
    },
    "relationships": {
        "products_sales": {
            "source_table": "sales",
            "target_table": "products",
            "source_column": "product_id",
            "target_column": "id",
            "relationship_type": "many-to-one",
            "description": "Each sale record references a product from the products table."
        },
        "products_categories": {
            "source_table": "products",
            "target_table": "categories",
            "source_column": "category_id",
            "target_column": "id",
            "relationship_type": "many-to-one",
            "description": "Each product belongs to a single category from the categories table."
        },
        "products_imports": {
            "source_table": "imports",
            "target_table": "products",
            "source_column": "product_id",
            "target_column": "id",
            "relationship_type": "many-to-one",
            "description": "Each import record references a product from the products table."
        }
    }
}

example_queries = {
    "queries": [
        {
            "user_query": "Show me the latest imported products.",
            "sql_query": "SELECT p.product_name, i.import_date, i.quantity, i.supplier FROM imports i JOIN products p ON i.product_id = p.id ORDER BY i.import_date DESC LIMIT 10;"
        },
        {
            "user_query": "How much revenue was generated by each category last month?",
            "sql_query": "SELECT c.category_name, SUM(s.amount) as total_revenue FROM sales s JOIN products p ON s.product_id = p.id JOIN categories c ON p.category_id = c.id WHERE s.sale_date BETWEEN '2023-08-01' AND '2023-08-31' GROUP BY c.category_name;"
        },
        {
            "user_query": "What is the total quantity of each product imported this year?",
            "sql_query": "SELECT p.product_name, SUM(i.quantity) as total_imported FROM imports i JOIN products p ON i.product_id = p.id WHERE i.import_date BETWEEN '2023-01-01' AND '2023-12-31' GROUP BY p.product_name;"
        },
        {
            "user_query": "Which products have not been sold in the last 6 months?",
            "sql_query": "SELECT p.product_name FROM products p WHERE p.id NOT IN (SELECT s.product_id FROM sales s WHERE s.sale_date >= NOW() - INTERVAL 6 MONTH);"
        },
        {
            "user_query": "Show me the top 5 products by revenue for the current year.",
            "sql_query": "SELECT p.product_name, SUM(s.amount) as total_revenue FROM sales s JOIN products p ON s.product_id = p.id WHERE s.sale_date BETWEEN '2023-01-01' AND '2023-12-31' GROUP BY p.product_name ORDER BY total_revenue DESC LIMIT 5;"
        }
    ]
}

feedback_data = [
    {
        "query": "Show me the total sales for each product.",
        "feedback": "The query was correct, but the format could be improved.",
        "sql_query": "SELECT product_name, SUM(amount) as total_sales FROM sales JOIN products ON sales.product_id = products.id GROUP BY product_name;"
    },
    {
        "query": "Show me the top 5 categories by revenue.",
        "feedback": "The generated query missed some categories.",
        "sql_query": "SELECT category_name, SUM(sales.amount) as total_revenue FROM sales JOIN products ON sales.product_id = products.id JOIN categories ON products.category_id = categories.id GROUP BY category_name ORDER BY total_revenue DESC LIMIT 5;"
    }
]


In [4]:
# Step 1: Metadata Retriever
class MetadataRetriever:
    def __init__(self, metadata):
        self.metadata = metadata
    
    def retrieve_metadata(self, query):
        # Retrieve relevant tables based on query
        relevant_tables = []
        for table in self.metadata["tables"]:
            if table in query:
                relevant_tables.append(table)
        return relevant_tables

In [5]:

# Step 2: Llama Model for NL2SQL Conversion
class LlamaNL2SQL:
    def __init__(self, model_name="llama-8b"):
        # self.client = OllamaClient(model_name=model_name)
        self.client = OllamaLLM(model=model_name)

    def query_to_sql(self, prompt):
        response = self.client.generate(prompt)
        return response['query']
    
    def get_embedding(self, text):
        # Placeholder for embedding generation
        return np.random.rand(128)  # Example embedding

In [14]:
# Step 3: RAG Query Chain
class RAGQueryChain:
    def __init__(self):
        self.retriever = MetadataRetriever(metadata)
        self.generator = LlamaNL2SQL()

    def get_sql_query(self, user_query):
        return "hi"
        # Step 1: Retrieve relevant context based on the user query
        query_embedding = self.generator.get_embedding(user_query)
        context_indices = self.retriever.retrieve_metadata(user_query)
        
        # Step 2: Use the retrieved context to generate the SQL query
        context = self.format_context(context_indices)
        sql_query = self.generator.query_to_sql(user_query + " " + context)
        
        return sql_query
    
    def format_context(self, context_indices):
        # Convert retrieved indices to context strings (e.g., table names, schema info)
        context = " ".join([str(index) for index in context_indices])
        return context

In [7]:
# Step 4: Database Connector
class DBConnector:
    def __init__(self, config):
        self.engine = sqlalchemy.create_engine(
            f"{config['dialect']}://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"
        )

    def execute_query(self, query):
        try:
            with self.engine.connect() as connection:
                result = pd.read_sql(query, connection)
            return result
        except Exception as e:
            return str(e)

In [8]:
# Step 5: Error Handling
class ErrorHandler:
    def parse_error(self, error_message):
        # Interpret SQL errors
        if "syntax error" in error_message:
            return "Syntax Error", "Check the SQL syntax."
        elif "no such table" in error_message:
            return "Missing Table", "Check the table name in your query."
        return "Unknown Error", "An unknown error occurred."

    def suggest_fix(self, error_type, query):
        # Suggest fixes based on error type
        if error_type == "Syntax Error":
            return "Try rephrasing the query or correcting SQL syntax."
        elif error_type == "Missing Table":
            return f"Make sure the table exists in the database: {query}"
        return "Try modifying the query."

In [9]:
# Step 6: Report Formatter
class ReportFormatter:
    def format_report(self, data):
        # Convert data into a human-readable format
        return data.to_string(index=False)


In [10]:
# Step 7: Feedback Handler
class FeedbackHandler:
    def __init__(self, feedback_data):
        self.feedback_data = feedback_data

    def store_feedback(self, query, feedback, sql_query):
        self.feedback_data.append({"query": query, "feedback": feedback, "sql_query": sql_query})
        print("Feedback stored:", {"query": query, "feedback": feedback, "sql_query": sql_query})


In [11]:
# Step 8: Report Generation Chain
class ReportChain:
    def __init__(self):
        self.rag_chain = RAGQueryChain()
        self.db_connector = DBConnector(config=db_config)
        self.error_handler = ErrorHandler()
        self.report_formatter = ReportFormatter()
        self.feedback_handler = FeedbackHandler(feedback_data)

    def generate_report(self, user_query):
       
        sql_query = self.rag_chain.get_sql_query(user_query)
        return sql_query
        result = self.db_connector.execute_query(sql_query)

        if isinstance(result, str):  # An error occurred
            error_type, error_message = self.error_handler.parse_error(result)
            suggestion = self.error_handler.suggest_fix(error_type, sql_query)
            return {"error": error_message, "suggestion": suggestion}
        
        formatted_report = self.report_formatter.format_report(result)
        return {"report": formatted_report, "sql_query": sql_query}

    def store_feedback(self, query, feedback, sql_query):
        self.feedback_handler.store_feedback(query, feedback, sql_query)


In [12]:
report_chain = ReportChain()

In [15]:
prompt = "How many user do you have"
response = report_chain.generate_report(prompt)

ValueError: Argument 'prompts' is expected to be of type List[str], received argument of type <class 'str'>.